In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

## Preprocessing the Data

In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'my_dataset\\train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=8,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 6382 files belonging to 104 classes.


In [3]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'my_dataset\\test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=8,
    image_size=(224, 224),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 1337 files belonging to 104 classes.


## Training the Model

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout

hub_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
base_model = hub.KerasLayer(hub_url, trainable=True)

input_tensor = tf.keras.layers.Input(shape=(224, 224, 3))
base_model_output = base_model(input_tensor)
x = Dropout(0.6)(base_model_output)
x = Dense(150, activation='relu')(x)
output = Dense(104, activation='softmax')(x)  

model = Model(inputs=input_tensor, outputs=output)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 keras_layer (KerasLayer)    (None, 1001)              3540265   
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 150)               150300    
                                                                 
 dense_1 (Dense)             (None, 104)               15704     
                                                                 
Total params: 3,706,269
Trainable params: 3,672,157
Non-trainable params: 34,112
_________________________________________________________________


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

callbacks = [early_stopping]

In [6]:
training_history = model.fit(x=training_set,validation_data=test_set,epochs=50,callbacks=callbacks)

Epoch 1/50
798/798 [==============================] - 116s 111ms/step - loss: 2.6118 - accuracy: 0.4331 - val_loss: 1.5350 - val_accuracy: 0.6425
Epoch 2/50
798/798 [==============================] - 77s 97ms/step - loss: 1.2947 - accuracy: 0.7031 - val_loss: 1.0905 - val_accuracy: 0.7883
Epoch 3/50
798/798 [==============================] - 76s 95ms/step - loss: 0.9660 - accuracy: 0.7791 - val_loss: 1.2382 - val_accuracy: 0.7861
Epoch 4/50
798/798 [==============================] - 77s 96ms/step - loss: 0.7727 - accuracy: 0.8196 - val_loss: 1.1213 - val_accuracy: 0.8175
Epoch 5/50
798/798 [==============================] - 72s 90ms/step - loss: 0.6881 - accuracy: 0.8450 - val_loss: 1.4745 - val_accuracy: 0.7973


## Evaluating the Model

In [7]:
val_loss, val_acc = model.evaluate(training_set)
print('Training accuracy:', val_acc)

798/798 [==============================] - 21s 26ms/step - loss: 0.6142 - accuracy: 0.8654
Training accuracy: 0.8654026985168457


In [8]:
val_loss, val_acc = model.evaluate(test_set)
print('Validation accuracy:', val_acc)

168/168 [==============================] - 4s 26ms/step - loss: 1.0905 - accuracy: 0.7883
Validation accuracy: 0.7883321046829224


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns


class_names = test_set.class_names
num_classes = len(class_names)

class_to_index = dict(zip(class_names, range(num_classes)))

y_true = []
for images, labels in test_set:
    y_true.extend(np.argmax(labels.numpy(), axis=1))

y_pred = model.predict(test_set)

y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

## Save the Model

In [14]:
model.save("model_batik_V2", save_format='tf')

loaded_model = tf.keras.models.load_model("model_batik_V2", custom_objects={'KerasLayer': hub.KerasLayer})

INFO:tensorflow:Assets written to: model_batik_V2\assets


INFO:tensorflow:Assets written to: model_batik_V2\assets


## Convert Model to TFJS

In [ ]:
import tensorflowjs as tfjs

model = tf.keras.models.load_model("/content/model_batik/")

tfjs.converters.tf_saved_model_conversion_v2.convert_keras_model_to_graph_model(model, "./tfjs_model")

## Convert Model to TFlite

In [ ]:
model_directory = 'model_batik_V2'

converter = tf.lite.TFLiteConverter.from_saved_model(model_directory)

tflite_model = converter.convert()

with open('modelV2.tflite', 'wb') as f:
    f.write(tflite_model)